In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
## Run Evaluation Script
# %matplotlib inline
# %run /media/dean/datastore1/datasets/Scripts/darknet_evaluate_bdd.ipynb

In [3]:
def parse_model_config(path):
    """Parses the yolo-v3 layer configuration file and returns module definitions"""
    file = open(path, 'r')
    lines = file.read().split('\n')
    lines = [x for x in lines if x and not x.startswith('#')]
    lines = [x.rstrip().lstrip() for x in lines] # get rid of fringe whitespaces
    module_defs = []
    for line in lines:
        if line.startswith('['): # This marks the start of a new block
            module_defs.append({})
            module_defs[-1]['type'] = line[1:-1].rstrip()
            if module_defs[-1]['type'] == 'convolutional':
                module_defs[-1]['batch_normalize'] = 0
        else:
            key, value = line.split("=")
            value = value.strip()
            module_defs[-1][key.rstrip()] = value.strip()

    return module_defs


def save_model_config(model_defs, path):
    """Saves the yolo-v3 layer configuration file"""
    with open(path, 'w') as writer:
        
        for block in model_defs:
            writer.write('['+ block['type'] +']'+'\n')
            [writer.write(str(k)+'='+str(v)+'\n') for k,v in block.items() if k != 'type']
            writer.write('\n')
    

def parse_data_config(path):
    """Parses the data configuration file"""
    options = dict()
    options['gpus'] = '0,1'
    with open(path, 'r') as fp:
        lines = fp.readlines()
    for line in lines:
        line = line.strip()
        if line == '' or line.startswith('#'):
            continue
        key, value = line.split('=')
        options[key.strip()] = value.strip()
    return options

In [4]:
import os
import yaml
import urllib
from PIL import Image
from enum import Enum
from pycocotools.coco import COCO


import xml.etree.cElementTree as ET
import glob
import argparse
import numpy as np
import json
import numpy
import cv2
from collections import OrderedDict
import scipy.misc
from skimage import measure   
import random
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import shutil
import pickle
import pandas as pd

from utils import datasets

WORKING_DIRECTORY = '/media/dean/datastore1/datasets/darknet'
DATA_DIRECTORY = os.path.join(WORKING_DIRECTORY, 'data')
BDD100K_DIRECTORY = os.path.join('/media/dean/datastore1/datasets/BerkeleyDeepDrive', 'bdd100k')
COCO_DIRECTORY = os.path.join(DATA_DIRECTORY, 'coco')
BDD100K_COCO_ANNOTATIONS_FILE = os.path.join(COCO_DIRECTORY,'annotations' , 'bdd100k_instances_train2014.json')



CATEGORY_NAMES = os.path.join(WORKING_DIRECTORY, 'data', 'coco.bdd100k.names')
TRAINERS_DIRECTORY = os.path.join(WORKING_DIRECTORY, 'trainers')
ANNOTATIONS_FILE = os.path.join(BDD100K_DIRECTORY, 'labels/kache_bdd100k_labels_images_train.json', )

DATA_CONFIG_PATH = os.path.join(WORKING_DIRECTORY, 'cfg', 'bdd100k.data')
MODEL_CONFIG_PATH = os.path.join(WORKING_DIRECTORY, 'cfg', 'yolov3-bdd100k.cfg')

In [5]:
# For Run in Training Runs    
trainers = os.listdir(TRAINERS_DIRECTORY)
all_training_runs = []

for trainer in trainers:
    ## Prepare Dataset ##
    bdd_set = datasets.DataFormatter(annotations_list = ANNOTATIONS_FILE, data_format = datasets.Format.bdd, 
                                     output_path = os.path.join(TRAINERS_DIRECTORY,trainer,'data'),
                                     trainer_prefix = 'COCO_train2014_0000', s3_bucket = 'kache-scalabel/bdd100k/images/100k/train/')
#     print(list(bdd_set._images.values())[:5],"\n"*4)
    
    # TODO: Export Dataset to darknet format
    bdd_set.export(datasets.Format.darknet)
    
    
    # TODO: Generate data config in trainers directory.
    
    print('Initiating Trainer:', os.path.join(TRAINERS_DIRECTORY,trainer))
    # Grab hyperparameters from filename
    tokens = trainer.split('_')
    hyperparams = {'name': tokens[0], 
                   'gpus': int(tokens[1].replace('gpu','')),
                   'lr': float('0.'+tokens[2].replace('lr','')),
                   'batch': int(tokens[3].replace('bat','')),
                   'subdivisions': int(tokens[4].replace('sd','')),
                   'epochs': int(tokens[5].replace('ep',''))}
    print('\t', 'Hyperparameters:', hyperparams)
    
    # Update model config
    
    model_config = parse_model_config(MODEL_CONFIG_PATH)
    for block in model_config:        
        if block['type'] == 'net':
            block['learning_rate'] = hyperparams['lr']
            block['batch'] = hyperparams['batch']
            block['subdivisions'] = hyperparams['subdivisions']
            block['burn_in'] = len(bdd_set._images)//(hyperparams['gpus'] * hyperparams['batch'])
            block['max_batches'] = len(bdd_set._images)//(hyperparams['gpus'] * hyperparams['batch']) * hyperparams['epochs']

    os.makedirs(os.path.join(TRAINERS_DIRECTORY,trainer, 'cfg'), exist_ok = True)
    save_model_config(model_config, os.path.join(TRAINERS_DIRECTORY,trainer, 'cfg', os.path.split(MODEL_CONFIG_PATH)[-1]))
    
    # Update data config
    data_config = parse_data_config(DATA_CONFIG_PATH)
    print(data_config)
    

69863
Length of Coco Annotations: 1286871
Initiating Trainer: /media/dean/datastore1/datasets/darknet/trainers/run6_2gpu_001lr_256bat_64sd_60ep
	 Hyperparameters: {'name': 'run6', 'gpus': 2, 'lr': 0.001, 'batch': 256, 'subdivisions': 64, 'epochs': 60}
{'gpus': '0,1', 'classes': '10', 'train': '/media/anthony/deans_data/darknet/data/coco/labels/train2014/manifast.txt', 'valid': '/media/anthony/deans_data/darknet/data/coco/labels/train2014/image_list.txt', 'names': '/media/anthony/deans_data/darknet/data/coco.bdd100k.names', 'backup': '/media/anthony/deans_data/darknet/backup', 'eval': 'coco'}


In [6]:
bdd_set.export(format = datasets.Format.darknet)